# Motif Enrichment
- The motif hits matrix save the max loglikelyhood score of a motif in that DMR
- Before run the motif enrichment, I filter the scores for each motif using cutoff defined by
    - `np.quantile(motif_score[motif_score > 0]) * mask_quantile_to99`
- This filter will remove hits with low score, which is likely to be false positive
- Its likely boosting the significance of real enrichment

In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
from scipy.sparse import csr_matrix
from statsmodels.stats.multitest import multipletests
import pathlib

In [2]:
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = 'IT-L23+ACA'

motif_hits_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifScan.h5ad'
dmr_hits_path = f'/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/HypoDMR/{group}.DMS1.bed'
output_dir = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifEnrichment'


In [3]:
# Parameters
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = "IT-L5+AI"
motif_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L5/MotifScan.h5ad"
dmr_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L5/HypoDMR/IT-L5+AI.DMS1.bed"
output_dir = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L5/MotifEnrichment"


In [4]:
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## DMR hits

In [5]:
dmr_bed_df = pd.read_csv(dmr_hits_path, sep='\t', header=None, index_col=3)
dmr_bed_df.head()

,0,1,2
3,,,
DGm3c_9,chr1,3114936,3115276
DGm3c_18,chr1,3211017,3211017
DGm3c_20,chr1,3213139,3213139
DGm3c_21,chr1,3214603,3214603
DGm3c_31,chr1,3269697,3269697


In [6]:
motif_hits = anndata.read_h5ad(motif_hits_path)
motif_hits = motif_hits[dmr_bed_df.index, :].copy()
motif_ids = motif_hits.var_names
motif_hits.X = motif_hits.X.todense()
motif_hits

AnnData object with n_obs × n_vars = 33931 × 719 
    obs: 'chrom', 'start', 'end'

In [7]:
# mask small motif scores
motif_cutoff = pd.Series(np.apply_along_axis(lambda i: np.quantile(i[i>0], 0.99) * mask_quantile_to99, 
                                             0, motif_hits.X),
                         index=motif_hits.var_names)
# only keep value larger than the cutoff for each motif
motif_hits.X = np.multiply(motif_hits.X, (motif_hits.X > motif_cutoff.values[None, :]))

## Background Hits

In [8]:
background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/BackgroundDMR/MotifScan.h5ad'
)

In [9]:
dmr_bed = pybedtools.BedTool().from_dataframe(dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

In [10]:
# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, motif_ids].copy()

In [11]:
judge_matrix = csr_matrix(background_motif_hits.X > motif_cutoff.values[None, :])
background_motif_hits.X = background_motif_hits.X.multiply(judge_matrix)

In [12]:
background_motif_hits

AnnData object with n_obs × n_vars = 339454 × 719 
    obs: 'chrom', 'start', 'end'

## Motif hits contingency table

In [13]:
# calculate motif occurence, not considering hits here
pos = (motif_hits.X > 0).sum(axis=0)
pos_total = motif_hits.shape[0]

neg = (background_motif_hits.X > 0).sum(axis=0)
neg_total = background_motif_hits.shape[0]

In [14]:
tables = {}
for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
    table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
    tables[motif] = table

In [15]:
results = {}
with ProcessPoolExecutor(20) as executor:
    fs = {}
    for motif, t in tables.items():
        f = executor.submit(stats.fisher_exact, t, alternative='greater')
        fs[f] = motif

    for f in as_completed(fs):
        motif = fs[f]
        odds, p = f.result()
        results[motif] = {'oddsratio': odds, 'p_value': p}
motif_enrich_df = pd.DataFrame(results).T

_, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
motif_enrich_df['adj_p'] = p

motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
    -np.inf, -300)

records = {}
for motif, t in tables.items():
    tp, tn = t[0]
    fp, fn = t[1]
    tp_rate = tp / pos_total
    fp_rate = fp / neg_total
    records[motif] = dict(tp=tp,
                          tn=tn,
                          fp=fp,
                          fn=fn,
                          tp_rate=tp_rate,
                          fp_rate=fp_rate)
counts = pd.DataFrame(records).T
motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

In [16]:
motif_enrich_df['Group'] = group

In [17]:
# final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

42

In [18]:
motif_enrich_df.to_msgpack(output_dir / f'{group}.motif_enrichment.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [19]:
pybedtools.cleanup()

In [20]:
filtered_motif_df

,oddsratio,p_value,adj_p,-lgp,tp,tn,fp,fn,tp_rate,fp_rate,Group
MA0052.4,3.072571,2.343033e-122,1.295877e-120,119.887436,719.0,33212.0,2375.0,337079.0,0.021190,0.006997,IT-L5+AI
MA0091.1,2.390767,1.300614e-67,3.463486e-66,65.460487,597.0,33334.0,2524.0,336930.0,0.017595,0.007435,IT-L5+AI
MA0461.2,3.274182,5.074017e-176,1.216073e-173,172.915040,954.0,32977.0,2973.0,336481.0,0.028116,0.008758,IT-L5+AI
MA0497.1,3.185405,1.833178e-146,1.882936e-144,143.725165,821.0,33110.0,2622.0,336832.0,0.024196,0.007724,IT-L5+AI
MA0509.2,4.836696,3.781457e-126,2.471698e-124,123.607005,435.0,33496.0,909.0,338545.0,0.012820,0.002678,IT-L5+AI
MA0510.2,4.178169,1.514663e-34,3.300129e-33,32.481469,132.0,33799.0,317.0,339137.0,0.003890,0.000934,IT-L5+AI
MA0600.2,4.282289,6.474926e-36,1.454835e-34,33.837186,134.0,33797.0,314.0,339140.0,0.003949,0.000925,IT-L5+AI
MA0601.1,1.570433,1.073051e-26,2.143121e-25,24.668953,730.0,33201.0,4687.0,334767.0,0.021514,0.013807,IT-L5+AI
MA0602.1,1.662876,8.428756e-18,1.236791e-16,15.907704,374.0,33557.0,2260.0,337194.0,0.011022,0.006658,IT-L5+AI
MA0606.1,1.515267,2.306918e-68,6.379516e-67,66.195212,2372.0,31559.0,16042.0,323412.0,0.069907,0.047258,IT-L5+AI
